In [1]:
# default_exp core.config
# export
from pathlib import Path
from shutil import which
import yaml
import os
import time
import getpass
import secrets

In [11]:
# export
import hashlib, binascii, os

def _hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

def _verify_password(stored_password, provided_password):
    """Verify a stored password against one provided by user"""
    salt = stored_password[:64]
    stored_password = stored_password[64:]
    pwdhash = hashlib.pbkdf2_hmac('sha512',
                                  provided_password.encode('utf-8'),
                                  salt.encode('ascii'),
                                  100000)
    pwdhash = binascii.hexlify(pwdhash).decode('ascii')
    return pwdhash == stored_password

In [12]:
# export

def is_tool(name):
    """Check whether `name` is on PATH and marked as executable."""
    return which(name) is not None

def guess_editor():
    editors = ['notepad','nano','vi']
    return next(e for e in editors if (is_tool(e)))

class Config():
    config_file = Path(os.getenv('QS_CONFIG', f'~/.config/as.yaml')).expanduser()

    def __init__(self):
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        if not self.config_file.exists(): self.create_config()
        self.d = self._load()

    def __getattr__(self, k):
        return self.get(k)

    def get(self, k, default=None):
        if k=='d': raise AttributeError
        return self.d.get(k, default)


    def create_config(self):
        self.d = {'version': 1.0,
                  'projects': {},
                  'editor': guess_editor(),
                  'secret': secrets.token_hex(32),
                  'passwords': {}}

        self.save()

    def save(self):
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        with self.config_file.open('w') as f:
            yaml.dump(self.d, f)

    def set_password(self, user, password):
        self.d['passwords'][user] = _hash_password(password)
        self.save()

    def verify_password(self, user, password):
        return _verify_password(self.d['passwords'].get(user, ''), password)

    def _load(self):
        with self.config_file.open() as f:
            return yaml.load(f, Loader=yaml.FullLoader)



config = Config()

In [13]:
c = Config()
c.d

{'editor': 'nano',
 'hei': 'a44d09ad1634bab8fe506e7a1fe5c527c2adf3eefe6d106cf523f499d7d23857bdb5d643a60569eee89acfa4afe0426a3402b8e2806badb4ae6c7408f2a4fc8e09d146b73e4261ab03f69c12ff82eb4fe2a683f14618e81268e75f15d197d62d',
 'passwords': {'hei': 'f142e5ffdd159e682c5fd2783db186f7ccbd47a6784127d0120306a38d416ddca86b4b6c2bd4669f9fa75eddf82e82ff05e7d7e22e468db3202a3a83942b85f1b8a877a92f3f7b6a8e43169a06a2f6726de41e43ce7a4130001beac673a4d9c7'},
 'projects': {'myscrum': '/tmp/myscrum',
  'myscrum2': '/tmp/myscrum2',
  'scrum': '/tmp/scrum'},
 'secret': '2d8deb6fe090e701951fea06fcad129b5631818ca660759e299c98e7f3f6fbe1',
 'version': 1.0}

In [14]:
c.create_config()

In [15]:
c.e
